# About

test notebook to explore spacy and tfidf and cosine similarity

https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/


use beautiful soup to extract the text only

https://www.nltk.org/book/ch03.html

bag of words

https://medium.com/glose-team/%EF%B8%8F-fast-bag-of-words-using-spacy-and-cython-574c308a9ff3

cosine similarity

https://markhneedham.com/blog/2016/07/27/scitkit-learn-tfidf-and-cosine-similarity-for-computer-science-papers/

In [23]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

# Loads Data

In [3]:
p_questions = "/mnt/disks/disk-1-w210-data/data/interim/stackexchange/stackoverflow/PostQuestionsFiltered_V3_parsed.tsv"
questions = pd.read_csv(p_questions, delimiter = "\t", encoding = "utf-8")

/home/yangyq/anaconda3/envs/w210/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
p_answers = "/mnt/disks/disk-1-w210-data/data/interim/stackexchange/stackoverflow/PostAnswersFiltered_V3_parsed.tsv"
answers = pd.read_csv(p_answers, delimiter = "\t", encoding = "utf-8")

## sample

In [5]:
t1 = questions[questions["accepted_answer_id"] > 0]
questions_sample = t1.sample(600)

## corpus

In [6]:
# constructs corpus
# with question id, title, accepted answer id, answer body
def construct_corpus(questions, answers = answers):
    t1 = questions[["id", "title", "accepted_answer_id"]].rename(columns = {"id" : "q_id", "title" : "q_title"})
    t2 = answers[["id", "body"]].rename(columns = {"id" : "a_id", "body" : "a_body"})
    t3 = t1.merge(t2, left_on = "accepted_answer_id", right_on = "a_id", how = "inner").drop(columns = "a_id")
    return(t3)

In [7]:
corpus = construct_corpus(questions_sample)
corpus.head(3)

,q_id,q_title,accepted_answer_id,a_body
0,1279779,What is the difference between \r and \n?,1279802.0,"<p>They're different characters. <code>\r</code> is carriage return, and <code>\n</code> is line feed.</p>\r\n\r\n<p>On ""old"" printers, <code>\r</code> sent the print head back to the start of the line, and <code>\n</code> advanced the paper by one line. Both were therefore necessary to start printing on the next line.</p>\r\n\r\n<p>Obviously that's somewhat irrelevant now, although depending on the console you may still be able to use <code>\r</code> to move to the start of the line and overwrite the existing text.</p>\r\n\r\n<p>More importantly, Unix tends to use <code>\n</code> as a line separator; Windows tends to use <code>\r\n</code> as a line separator and Macs (up to OS 9) <em>used</em> to use <code>\r</code> as the line separator. (Mac OS X is Unix-y, so uses <code>\n</code> instead; there may be some compatibility situations where <code>\r</code> is used instead though.)</p>\r\n\r\n<p>For more information, see the <a href=""http://en.wikipedia.org/wiki/Newline"" rel=""noreferrer"">Wikipedia newline article</a>.</p>\r\n\r\n<p>EDIT: This is language-sensitive. In C# and Java, for example, <code>\n</code> <em>always</em> means Unicode U+000A, which is defined as line feed. In C and C++ the water is somewhat muddier, as the meaning is platform-specific. See comments for details.</p>"
1,28015965,"Editing an element's style based on the ""value"" attribute of its child element",28015976.0,"<ul>\r\n<li>In your example, <code>this</code> doesn't refer to each element in your jQuery object. You can either loop through the object or, since <a href=""http://api.jquery.com/css/"" rel=""nofollow""><code>.css()</code></a> handles looping for you, pass a callback function as the second parameter. The context of this function will be each element in the object.</li>\r\n<li>jQuery has <a href=""http://api.jquery.com/val/"" rel=""nofollow""><code>.val()</code></a> to get the value property of form elements.</li>\r\n</ul>\r\n\r\n\r\n\r\n<pre><code>$("".color-button"").css(""background-color"", function(){\r\n return $(this).find('input').val();\r\n});\r\n</code></pre>\r\n\r\n<p><a href=""http://jsfiddle.net/eqLo24pu/"" rel=""nofollow""><strong>JSFiddle</strong></a></p>"
2,43711355,Creating a color bar for a graph that does not contain a mappable?,43711456.0,"<p>You can of course create your own ScalarMappable, </p>\r\n\r\n<pre><code>sm = plt.cm.ScalarMappable(cmap=...)\r\n</code></pre>\r\n\r\n<p>which would then need to have an array mapping to colors (<code>sm.set_array()</code>).</p>\r\n\r\n<pre><code>import matplotlib.pyplot as plt\r\nimport matplotlib.colors\r\nimport numpy as np\r\n\r\ncolorDict = {'red': [(0.0, 1.0, 1.0), (1.0, 0.0, 0.0)], 'green': [(0.0, 0.0, 0.0), (1.0, 1.0, 1.0)], 'blue': [(0.0, 0.0, 1.0), (1.0, 0.0, 0.0)]}\r\nrgColorMap = matplotlib.colors.LinearSegmentedColormap('RedGreen', colorDict, 10)\r\nseedColors = plt.cm.get_cmap(rgColorMap, 10)\r\n\r\nsm = plt.cm.ScalarMappable(cmap=seedColors)\r\nsm.set_array(range(1, 9))\r\nfig = plt.figure(figsize=(8.27, 11.69), dpi=100)\r\nfor lineInd in range(1, 9):\r\n plt.plot([x for x in range(1, 10)], [y*lineInd for y in range(1, 10)], color=seedColors(lineInd))\r\n\r\n\r\nfig.colorbar(sm, ticks=range(1,9), boundaries=np.arange(0.5,9,1))\r\n\r\nplt.show()\r\n</code></pre>\r\n\r\n<p><a href=""https://i.stack.imgur.com/ref3O.png"" rel=""nofollow noreferrer""><img src=""https://i.stack.imgur.com/ref3O.png"" alt=""enter image description here""></a></p>"


# Create and Train Model

## cleaner

In [8]:
from sklearn.base import TransformerMixin
from bs4 import BeautifulSoup

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # gets just the txt, without the html tags
    t1 = BeautifulSoup(text, 'html.parser').get_text()
    # Removing spaces and converting text into lowercase
    t2 = t1.strip().lower()
    return t2

In [20]:
# tests on the cleaner
t1 = predictors()
t2 = corpus.iloc[1,3]
print("===========")
print(t2)
print("===========")
print(clean_text(t2))
print("===========")
print(t1.transform([t2]))

<ul>
<li>In your example, <code>this</code> doesn't refer to each element in your jQuery object. You can either loop through the object or, since <a href="http://api.jquery.com/css/" rel="nofollow"><code>.css()</code></a> handles looping for you, pass a callback function as the second parameter. The context of this function will be each element in the object.</li>
<li>jQuery has <a href="http://api.jquery.com/val/" rel="nofollow"><code>.val()</code></a> to get the value property of form elements.</li>
</ul>



<pre><code>$(".color-button").css("background-color", function(){
    return $(this).find('input').val();
});
</code></pre>

<p><a href="http://jsfiddle.net/eqLo24pu/" rel="nofollow"><strong>JSFiddle</strong></a></p>
in your example, this doesn't refer to each element in your jquery object. you can either loop through the object or, since .css() handles looping for you, pass a callback function as the second parameter. the context of this function will be each element in the obje

## tokenizer

In [22]:
import string
import spacy

# Create our list of punctuation marks
# TODO more punctuations???
punctuations = string.punctuation

# Create our list of stopwords
# TODO en_core_web_sm ???
nlp = spacy.load("en")
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = spacy.lang.en.English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

## vectorizer

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TODO tweak min doc frequency???
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0)

## train model

In [27]:
from sklearn.pipeline import Pipeline

# Create pipeline using a custom cleaner, and our vectorizer
pipe = Pipeline([
    ("cleaner", predictors()),
    ('vectorizer', vectorizer)
])

# model generation
m1 = pipe.fit_transform(corpus.q_title)

In [28]:
# inspect tokens
tokens = vectorizer.get_feature_names()
print(len(tokens))
print(np.asarray(tokens[1:100]))

9424
['10 12' '10 12 system' '10 android' '10 android runtime' '11' '11 bar'
 '11 bar graphs' '12' '12 system' '120' '120 170' '120 170 px' '150'
 '150 markers' '150 markers in' '170' '170 px' '1px' '1px dotted'
 '1px dotted border' '20' '20 frames' '20 frames per' '2003' '2003 2007'
 '2003 2007 do' '2007' '2007 do' '2007 do not' '2010' '2010 release'
 '2011' '2015' '2015 debugger' '2015 debugger visualizers' '2017'
 '2017 naming' '2017 naming convention' '2d' '2d graphics'
 '2d graphics library' '2d humanoid' '2d humanoid stickman'
 '2d optionally' '2d optionally 3d' '32' '32 32' '32 32 px' '32 bit'
 '32 bit imul' '32 px' '32 px to' '365' '365 using' '365 using work' '3d'
 '3d cube' '3d cube won' '3d drawing' '3d drawing library' '3d models'
 '3d models from' '3d occlusion' '3d occlusion culling' '3d points'
 '3d rotation' '3d rotation matrix' '3d space' '3d space using' '40'
 '40 words' '40 words of' '50' '50 2010' '50 2010 release' '64' '64 bit'
 '64 bit result' '90' '90 degs' '90 d

In [46]:
from sklearn.metrics.pairwise import linear_kernel

# helper function to compare a input question to every question in the corpus
def find_similar(vectorizer, tfidf_matrix, question_input, top_n = 3):
    t1 = vectorizer.transform([question_input])
    cosine_similarities = linear_kernel(t1, tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1]]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

In [35]:
# helper function to decorate the results found by the find_similar function
def decorate_answers(similar_qs, corpus = corpus):
    t1 = pd.DataFrame(similar_qs, columns = ["q_index", "cos_similarity"]).set_index(["q_index"])
    t2 = corpus.iloc[t1.index]
    t3 = pd.concat([t1, t2], axis = 1)
    return(t3)

# match query

In [53]:
# match on a question that is an exact match of an existing question
# should contain at least 1 match with cosine similarity == 1
similar_qs = find_similar(
    vectorizer = vectorizer
    , tfidf_matrix = m1
    , question_input = "What is the difference between \r and \n?"
)
similar_qs

[(0, 1.0), (76, 0.21873194315878874), (99, 0.17297366660617064)]

In [54]:
decorate_answers(similar_qs)

,cos_similarity,q_id,q_title,accepted_answer_id,a_body
0,1.000000,1279779,What is the difference between \r and \n?,1279802.0,"<p>They're different characters. <code>\r</code> is carriage return, and <code>\n</code> is line feed.</p>\r\n\r\n<p>On ""old"" printers, <code>\r</code> sent the print head back to the start of the line, and <code>\n</code> advanced the paper by one line. Both were therefore necessary to start printing on the next line.</p>\r\n\r\n<p>Obviously that's somewhat irrelevant now, although depending on the console you may still be able to use <code>\r</code> to move to the start of the line and overwrite the existing text.</p>\r\n\r\n<p>More importantly, Unix tends to use <code>\n</code> as a line separator; Windows tends to use <code>\r\n</code> as a line separator and Macs (up to OS 9) <em>used</em> to use <code>\r</code> as the line separator. (Mac OS X is Unix-y, so uses <code>\n</code> instead; there may be some compatibility situations where <code>\r</code> is used instead though.)</p>\r\n\r\n<p>For more information, see the <a href=""http://en.wikipedia.org/wiki/Newline"" rel=""noreferrer"">Wikipedia newline article</a>.</p>\r\n\r\n<p>EDIT: This is language-sensitive. In C# and Java, for example, <code>\n</code> <em>always</em> means Unicode U+000A, which is defined as line feed. In C and C++ the water is somewhat muddier, as the meaning is platform-specific. See comments for details.</p>"
76,0.218732,19610230,"What is a ""brush""?",19610465.0,"<p>OK, I found what I was looking for, pretty much:</p>\r\n\r\n<blockquote>\r\n <p>Brushing means selecting a subset of the data items with an input device (mouse). This is usually done to highlight this subset, but it can also be done to delete it from the view or to de-emphasize it, if the user wants to focus on the other items.</p>\r\n</blockquote>\r\n\r\n<p>Original <a href=""http://old.vrvis.at/via/resources/DA-RVoigt/node10.html"" rel=""noreferrer"">here</a>.</p>"
99,0.172974,48626591,Order bars by difference between variables,48627032.0,"<p>First of all, you shouldn't include <code>diff</code> (your result column) when using <code>gather</code>, it complicates things.<br>\r\nChange line <code>gather(na_item, values, 2:7)</code> to <code>gather(na_item, values, 2:6)</code>.</p>\r\n\r\n<p>You can use this code to calculate difference and order (using <code>dplyr::arange</code>) rows in descending order:</p>\r\n\r\n<pre><code>plotData &lt;- data_1_4 %&gt;% \r\n spread(na_item, values) %&gt;% \r\n na.omit() %&gt;% \r\n mutate(HH_FIN_EX = P31_S14_S15 / B1GQ, \r\n ACT_IND_CON_EXP = P41 / B1GQ, \r\n diff = ACT_IND_CON_EXP - HH_FIN_EX) %&gt;%\r\n gather(na_item, values, 2:6) %&gt;%\r\n filter(na_item %in% c(""HH_FIN_EX"", ""ACT_IND_CON_EXP"")) %&gt;%\r\n arrange(desc(diff))\r\n</code></pre>\r\n\r\n<p>And plot it with: </p>\r\n\r\n<pre><code>ggplot(plotData, aes(geo, values, fill = na_item))+\r\n geom_bar(stat = ""identity"", position = ""dodge"", color = ""black"") +\r\n labs(x = ""Countries"", \r\n y = ""As percentage of GDP"") +\r\n scale_x_discrete(limits = plotData$geo)\r\n</code></pre>\r\n\r\n<p><a href=""https://i.stack.imgur.com/Wwlds.png"" rel=""nofollow noreferrer""><img src=""https://i.stack.imgur.com/Wwlds.png"" alt=""enter image description here""></a></p>"


In [50]:
# match on a question that is not an existing question
similar_qs = find_similar(
    vectorizer = vectorizer
    , tfidf_matrix = m1
    , question_input = "How do I create a bar chart?"
)
similar_qs

[(421, 0.22415787077417781),
 (158, 0.21872599155408767),
 (370, 0.2042866573746666)]

In [51]:
decorate_answers(similar_qs)

,cos_similarity,q_id,q_title,accepted_answer_id,a_body
421,0.224158,13824612,How do Loop this command?,13824655.0,"<p>Use a loop, be it a <code>for</code> or a <code>while</code>. Using <code>goto</code> for looping is not used anymore in any <em>modern</em> language that I can think of! That kind of iteration is used in assembler, or in very primitive scripting/programming languages. Haven't you coded anything in the last, say, 20 years or so? :P</p>"
158,0.218726,7947150,How do you create multi colored pins in a map?,7948924.0,"<p>Try changing:</p>\r\n\r\n<pre><code>if (tempPin.pinType == @""VENUE"") {\r\n</code></pre>\r\n\r\n<p>to:</p>\r\n\r\n<pre><code>if ([((mapPinAnnotation*)annotation).pinType isEqualToString:@""VENUE""]) {\r\n</code></pre>"
370,0.204287,54513904,Switch between pie charts and bar chart in highcharts,54514421.0,"<p>You can add a custom item to the exporting menu and in <code>click</code> event use <code>update</code> method for the chart with new options:</p>\r\n\r\n<pre><code>exporting: {\r\n menuItemDefinitions: {\r\n // Custom definition\r\n switchChart: {\r\n onclick: function() {\r\n var chartType = this.options.chart.type;\r\n\r\n this.update({\r\n chart: {\r\n type: chartType === 'bar' ? 'pie' : 'bar'\r\n }\r\n })\r\n },\r\n text: 'Switch chart'\r\n }\r\n },\r\n buttons: {\r\n contextButton: {\r\n menuItems: [""switchChart"", ""separator"", ""printChart"", ""separator"", ""downloadPNG"", ""downloadJPEG"", ""downloadPDF"", ""downloadSVG""]\r\n }\r\n }\r\n},\r\n</code></pre>\r\n\r\n<p>Live demo: <a href=""http://jsfiddle.net/BlackLabel/xdsgL6rm/"" rel=""nofollow noreferrer"">http://jsfiddle.net/BlackLabel/xdsgL6rm/</a></p>\r\n\r\n<p>API: </p>\r\n\r\n<p><a href=""https://api.highcharts.com/highcharts/exporting.buttons.contextButton.menuItems"" rel=""nofollow noreferrer"">https://api.highcharts.com/highcharts/exporting.buttons.contextButton.menuItems</a></p>\r\n\r\n<p><a href=""https://api.highcharts.com/class-reference/Highcharts.Chart#update"" rel=""nofollow noreferrer"">https://api.highcharts.com/class-reference/Highcharts.Chart#update</a></p>"
